Links: 

https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#programming-model

https://github.com/syalanuj/youtube/blob/main/spark_streaming_with_python_in_12_minutes/spark_st_run.ipynb

https://github.com/Krupique/cursos-datascience-conteudo/blob/main/DSA_DS-02-BigData%20Analytics%20com%20Python%20e%20Spark/05%20-%20Introducao-SparkStreaming.ipynb


## Spark commands

## Project

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from collections import namedtuple

# Can only run this once. restart your kernel for any errors.

ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)
socket_stream = ssc.socketTextStream("127.0.0.1", 5554)

lines = socket_stream.window( 100 )

fields = ("text", "count" )

# Use Parenthesis for multiple lines or use \.
Tweet = namedtuple( 'Tweet', fields )
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(100).registerTempTable("tweets") ) ) # Registers to a table.

## Now run TweetListener.py

In [3]:
ssc.start()    

Py4JJavaError: An error occurred while calling o32.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "C:\Tools\Spark\python\pyspark\streaming\util.py", line 68, in call
    r = self.func(t, *rdds)
  File "C:\Tools\Spark\python\pyspark\streaming\dstream.py", line 161, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "C:\Users\krupc\AppData\Local\Temp/ipykernel_17784/3147682063.py", line 24, in <lambda>
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  File "C:\Tools\Spark\python\pyspark\sql\session.py", line 61, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "C:\Tools\Spark\python\pyspark\sql\session.py", line 605, in createDataFrame
    return self._create_dataframe(data, schema, samplingRatio, verifySchema)
  File "C:\Tools\Spark\python\pyspark\sql\session.py", line 628, in _create_dataframe
    rdd, schema = self._createFromRD

In [ ]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [ ]:
count = 0
while count < 10:
    
    time.sleep( 1 )
    top_10_tweets = sqlContext.sql( 'Select * from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    #sns.barplot(x='count',y='land_cover_specific', data=df, palette='Spectral')
    sns.barplot( x="count", y="text", data=top_10_df)
    plt.show()
    
    
    count = count + 1

In [4]:
top_10_tweets = sqlContext.sql( 'Select * from tweets' )
top_10_df = top_10_tweets.toPandas()
top_10_df.head(15)

,text,count
0,#6,1
1,#yell…rt,1
2,#izyum.,1
3,#puppy,1
4,#dogoftheday,1
5,#cutepuppies,1
6,#yellowcbd,1
7,#foreverlovepuppies,1
8,#jorin4eve,1
9,#4eve,1


In [6]:
ssc.stop()